# HOW TO CREATE A NESTED CONFIGURATION (eNEATL36 + BIZoo case)


This notebook is a guide to create a nested configuration with one nest. This is complementary to the IMMERSE deliverable 6.1 (available here : https://github.com/immerse-project/eNEATL36-BIzoo_Demonstator/blob/main/DOC/PDFs/RAPPORT_IMMERSE_D6p1.pdf), which contains many additional informations. In particular, we recommend to read the part 'II.C : Setup of the nested configuration' of the deliverable.


See also the AGRIF user guide for NEMO4.X versions:
https://forge.ipsl.jussieu.fr/nemo/chrome/site/doc/NEMO/guide/html/zooms.html 



### 1 - retrieve emodnet bathymetry

- *1: Download emodnet tiles (emodnet 2018 was used for BIZoo, but this should be similar with newer emodnet versions) => https://emodnet.ec.europa.eu/en*


- *2: A tool to merge emodnet tiles is available on the IMMERSE github: https://github.com/immerse-project/eNEATL36-BIzoo_Demonstator/tree/main/SCRIPTS/TOOLS/BATHY/MERGE_EMODNET


Step 1 : interpolate tiles on the full emodnet grid with 1_interp_tiles_on_full_emodnet.sh

Step 2 : merge tiles with 2_merge_tiles_2018.py


- *At this point, you should have a netcdf file with the emodnet bathymetry bathy_emodnet_on_EMODNET_2018.nc*

### 2 - Parent domain_cfg.nc file

NEMO4.X versions now need a file domain_cfg.nc that contains the coordinates of the configuration. The first step is to create it for the parent configuration only (e.g: eNEATL36)
- *use the DOMAINCFG tool in  NEMODIR/tools/DOMAINcfg, compiled without key_agrif*



### 3 - Create AGRIF_FixedGrids.in file


The AGRIF_FixedGrids.in file contains the information about the position of the nest in the parent grid. The file should contain at least three lines (for 1 zoom, and more if there is several nests) such as (for the BIZoo nest) :


`1
220 1073 350 1245 3 3 3
0
`

The first line indicates the number of zooms (1). The second line contains the starting and ending indices in both directions on the root grid (imin=220 imax=1073 jmin=350 jmax=1245) followed by the space and time refinement factors (3 3 3). The last line is the number of child grid nested in the refined region (0). A more complex example with telescoping grids can be found below and in the AGRIF_DEMO reference configuration directory.




### 4 - First creation of the domain_cfg.nc files for both parent and child grids

- *First, Compile the DOMAINCFG tool with the cpp key 'key_agrif'* added in cpp_DOMAINcfg.fcm file
- *Then, run the DOMAINCFG tool*

To run the tool with agrif, you should have one namelist per grid: in our case, one namelist 'namelist_cfg' for the parent grid, and another one 1_namelist_cfg for the child grid

 - *To do this, a tool is available "SHOW WHERE" : make_namelist.py, the tool creates both namelists.*
 
 - *Then, you need to update some parameters of the namelist. One of the most important is nn_bathy=2 if you want the bathymetry to be generated from an external bathymetry dataset such as emodnet, nn_bathy=1 if you already have the bathymetry file on the nest grid and nn_bathy=3 otherwise. In our case, with emodnet:*
 
 
`   ln_read_cfg = .false.   !  Read from a domain_cfg file
    nn_bathy    =    2      ! = 0 compute analyticaly
                           ! = 1 read the bathymetry file
                           ! = 2 compute from external bathymetry (should be on a regular grid)
                           ! = 3 compute from parent (if "key_agrif")
   cn_topo = 'emodNET_2018.nc' ! Name of the bathymetry file 
   cn_bath = 'bathymetry' ! Name of the bathymetry variable in cn_topo
   cn_lon = 'lon' ! Name of longitude in cn_topo
   cn_lat = 'lat' ! Name of latitude in cn_topo
`
 
- finally, run the domain cfg tool (a script is available here :https://github.com/immerse-project/eNEATL36-BIzoo_Demonstator/blob/main/SCRIPTS/DOMAINcfg/run_domain_cfg_eNEATL36_AGRIF_emodNET2018_finaldomain_corrected_with_FES2014.sub)

### 5 - Update the bathymetry with the BMG TOOL

External dataset bathymetry is interpolated on the nest grid with a classic bilinear interpolation. Thus, mismatches with the coastlines can occur. In order to refine the coaslines, you can download the BMG tool from IFREMER (https://mars3d.ifremer.fr/Les-outils/BathyMeshGridTOOLS) which is very useful to refine the bathymetry. 

- *Download bmg tool, or load bmg module on mercator px:*

`module load bmg/bmg-5.0 `

- *The tool is not maintained anymore but remains useful. It requires the data to be formatted a certain way to work. The script SCRIPT HERE is meant to create the coordinate file and the bathymetry file which are needed by BMG from the DOMAINCFG* 

`./make_domaincfg_compatible_BMG.sh DOMAIN_CFG_FILE_NAME OUT_COORD_FILE_NAME OUT_BATHY_FILE_NAME`

- *Download a .line coastline file, you can find the GSHHG coastline file here : =======PATH_COASTLINES=======

- *Then run CheckBMG*

`CheckBMG`

- *Go to 'files', then 'open files'. Select 'NEMO' as 'grid file format'. Then select the file OUT_COORD_FILE_NAME for the NEMO coordinates files, and OUT_BATHY_FILE_NAME for the bathymetry, and the .line file as coastline file.*



- *Then, have fun updating the bathymetry!*






Nb: depending on the bathymetry source, one might want to also add the tidal amplitude to the bathymetry (see IMMERSE Deliverable 6.1)


### 6 - Run domain_cfg a second time with the new updated bathymetry

- *Run the domaincfg tool once again while reading the new bathymetry file created at last step, with the following parameters:*

 
`   ln_read_cfg = .false.   !  Read from a domain_cfg file
    nn_bathy    =    2      ! = 0 compute analyticaly
                           ! = 1 read the bathymetry file
                           ! = 2 compute from external bathymetry (should be on a regular grid)
                           ! = 3 compute from parent (if "key_agrif")
   cn_topo = 'OUT_BATHY_FILE_NAME' ! Name of the bathymetry file 
   cn_bath = 'Bathymetry'
   cn_lon = 'glamt'
   cn_lat = 'gphit'
`
 
 
 
At this step, you will have both parent and child coordinate files domain_cfg.nc and 1_domain_cfg.nc respectively. 